### Topology Optimization

Let's take a look at a much more developed application of Devito. Here, we're going to be using Devito to solve some <a href="https://en.wikipedia.org/wiki/Topology_optimization">topology optimization</a> problems. These are the type of problems that Devito is well-suited for. Specifically, we'll be looking at an example that dolfin-adjoint looks at <a href="http://www.dolfin-adjoint.org/en/latest/documentation/poisson-topology/poisson-topology.html">here</a>, and rewriting it with Devito. Their example originally comes from <a href="Topology optimization of heat conduction problems using the finite volume method">Topology optimization of heat conduction problems using the finite volume method</a> by A. Gersborg-Hansen, M.P. Bendsøe, and O. Sigmund, which we'll reference in this article as "the paper".

Let's take look at what we want to produce, so we know where we're headed and what to compare ourselves against. We'll take the first of the test problems from the paper. Although we can't show it up here in this notebook, a quick glance at the paper itself shows that the team produced dentrite-like shapes throughout the structure when the boundary conditions for the left and top sides have the condition

$$ T = 0 $$

and the bottom and right sides have the condition 

$$ (k\Del T) \dot \textbf{n}=0 $$

We'll take the same conditions, and see if using the same equations, we can produce the same results by just working with Devito.

In [ ]:
from devito import Operator, TimeData, left, x, y
from matplotlib import pyplot
from scipy import integrate
import numpy

from devito.finite_difference import first derivative, left, right

# VARIABLE DECLARATIONS

# Create the 2d space for our problem
nx = 41
ny = 41
nt = 500
dx = 2. / (nx - 1) 
dy = 2. / (ny - 1)
x = numpy.linspace(0, 2, nx)
y = numpy.linspace(0, 2, ny)
X, Y = numpy.meshgrid(x, y)

# Initialize the TimeData object 'u'. TODO: Do we need this?
u = TimeData(name='u', shape=(nx,ny)) 
u.data.fill(0.)

# Define equations we'll use throughout our code
del_op = Del() # TODO: Check this usage of Del. Could be .gradient?
heat_flux = Eq(k * del_op(T)) # We'll use this as an abbreviation later

# TODO: Define other variables for the problem
temp = 0.
heat_conduction = 0.
vol_heat_source = 0.

# This is the Lagrange multiplier, lambda
lagrange_mult = 0.

Now we have a 2d space that we can work with.

Again, we now take equations from the paper we're looking at, giving us the following:

$$ \int_{\Omega_e} (\nabla \cdot (k \nabla T) + f) = 0 $$
$$ \implies \int_{\partial \Omega_e} (k \nabla T) \cdot \mathbf{n} + \int_{\Omega_e} f = 0 \ \forall_e $$

$$

In [ ]:
# SET UP THE PROBLEM

# Dirichlet boundary conditions
bc_u = [Eq(x.indexed[t+1, 0, y], 0.)]      # left
bc_u += [Eq(x.indexed[t+1, x, ny-1], 0.)]  # top

bc_u += [Eq(x.indexed[t+1, nx-1, y], 0.)]  # right
bc_u += [Eq(x.indexed[t+1, x, 0], 0.)]     # bottom

# Neumann boundary conditions

# Cost_function eqautions (these are equivalent)
cost_function1 = Eq(integrate(vol_heat_source * temp))
cost_function2 = Eq(integrate(del_op(temp).dot(heat_flux)))

# TODO
# unitvec
# equations

